# SBA DATA Act Pilot: JAAMS + Prism

The code below uses the JAAMS and Prism test data supplied by SBA to create a single file that contains the required DATA Act elements. This is a first pass, and this notebook tracks the process, assumptions, and outstanding questions.

In [7]:
#style up the notebook
#from IPython.core.display import HTML
#css_file = 'assets/css/notebook-style.css'
#HTML(open(css_file, "r").read())

In [6]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
pd.set_option('expand_frame_repr', True)
award_string = 'SBAHQ-'

## Find Files and Columns
Using a sample award id string, look through JAAMS and Prism files to see which ones contain award ids.
For files that contain award id data, find out which column(s) the award id is in.

In [562]:
#assuming that award ids for this data subset are start with 'SBAHQ-', 
#find the JAAMS and Prism tables that contain award is somewhere
#in their data
awards_files = !grep --include=\*.txt -rnl -e 'SBAHQ-'  data
jaams_awards_files = [f.split('\n')[0] for f in awards_files if f.find('jaams') > -1]
prism_awards_files = [f.split('\n')[0] for f in awards_files if f.find('prism') > -1]
prism = {}
#raw prism files aren't quoted, so many lines won't parse (skip them for now)
for file in prism_awards_files:
    key = file.split('/')[-1][:-4].lower()
    prism[key] = pd.read_csv(file, error_bad_lines = False, warn_bad_lines = False)
    prism[key].rename(columns=lambda x: '{}.'.format(key) + x.lower(), inplace = True)
prism_csv = pd.read_csv('data/data_act_prism_grants_fy14.csv')
jaams = {}
for file in jaams_awards_files:
    key = file.split('/')[-1][:-4].lower()
    jaams[key] = pd.read_csv(file, index_col = False)
    jaams[key].rename(columns=lambda x: '{}.'.format(key) + x.lower(), inplace = True)

### JAAMS

In [563]:
print 'looking for JAAMS colums that have an award id\n'
for k, df in jaams.iteritems():
    for col in df:
        if df[col].to_string().find(award_string) >= 0:
            print ('{}: {}'.format(k, col))

looking for JAAMS colums that have an award id

po_requisition_headers_all: po_requisition_headers_all.attribute2
po_lines_all: po_lines_all.attribute1
ap_invoices_all: ap_invoices_all.invoice_num
ap_invoices_all: ap_invoices_all.attribute11
po_headers_all: po_headers_all.segment1
ap_invoice_distributions_all: ap_invoice_distributions_all.attribute3


### Prism

In [564]:
print 'looking for Prism colums that have an award id\n'
for k, df in prism.iteritems():
    for col in df:
        if df[col].to_string().find(award_string) >= 0:
            print ('{}: {}'.format(k, col))
for col in prism_csv:
    if prism_csv[col].to_string().find(award_string) >= 0:
        print ('prism csv: {}'.format(col))

looking for Prism colums that have an award id

header: header.docnum
grantheader: grantheader.sba1222progcode
grantheader: grantheader.sba1222budgetcode
grantheader: grantheader.sba1222documentno
faadsciv: faadsciv.docnum
prism csv: Grant Number
prism csv: Award ID
prism csv: Unique Identifer


## Joining JAAMS and Prism

This work hinges on the ability to match SBA's financial system (JAAMS) and its grants system (Prism). The starting point was award id identified in the mapping document: JAAMS PO_HEADERS_ALL.SEGMENT1. We matched that to the Prism HEADER.DOCNUM.

This process yielded only a single matching award id between JAAMS and Prism, but we believe that's due to mis-matched fiscal years (see the questions below).

Once JAAMS and Prism were joined, we built out from there, pulling in other financial and grants tables using the provided files and SQL join statements.

The image below is a rough sketch of how we joined the tables to gather the required DATA Act information.

![SBA DATA Act Mappings](assets/images/jaams-prism-data-act-mapping.png)

## Assumptions, Questions and Missing Data

1. When matching JAAMS PO_HEADERS_ALL to Prism headers, how should the header.vernum be handled? What does it represent? Should we only be taking one vernum per docnum?
1. What does it mean when there are multiple records in PO_HEADERS_ALL with the same PO_HEADER_ID (the other data looks the same too)? For example, there are 9 records with a PO_HEADER_ID = 351260.
1. Missing Prism **faadsciv.census_code** (place of performance city). Used _countycityname_ column instead.
2. Missing Prism **docaddr** table (for awarding office name). Hard-coded _Office of Grants Management_, since that's the correct office for the one award we're tracking in this sprint.
3. Missing JAAMS **find_flex_values_vl** table (for funding offfice code). Hard-coded **`60200`**, since that's the correct funding office code for the one award we're tracking in this sprint.
4. Missing Prism **Vendor2** table (for recipient type). Hard-coded _12 Other Nonprofit_, since that's the correct type for the one award we're tracking in this sprint.
5. Why are **unobligated amount** and **amount of other budgetary resources** marked as NA in the mapping document?
6. Is there any more information available about **Type of Action** (it's marked as a computed column in the mapping document)?
7. Why are **awardee/recipient parent DUNS Number** and **awarding/recipient parent legal business name** marked as NA in the mapping document?
8. Why are **awardee/recipient country name and congressional district** marked as NA in the mapping document?
9. The mapping doc doesn't specify a mapping for **CFDA program number/title**. This code uses the following Prism fields:
    * faadsciv.programnumber
    * faadsciv.programtitle
10. The version of the mapping doc we're using says that TAS info isn't current in use (and that GTAS will be implemented soon). A prior version of the doc maps TAS to **Prism itemacct.TAS#**. For this sprint, we hard-coded the TAS that corresponds to the single award we matched. Going forward, should we look for TAS in itemacct until GTAS is implemented?
11. The mapping doc doesn't specify a mapping for **Place of Performance County Name** and **Place of Performance County Code**. For this sprint, we used the following Prism fields:
    * grantheader.sba1222countyname
    * grantheader.sba1222countycode
12. Why are **period of performance potential end date** and **ordering period end date** marked as NA in the mapping doc?
13. We might has mis-matched years in JAAMS and Prism. The Prism sql (data/prism/joins.txt) is grabbing records with a status date between 10/10/2014 and 10/1/2015 (i.e., FY 2015). Not sure if that's the code used to generate our sample data, but mismatched fiscal years could explain why we only matched a single award.
14. The mapping doc contains mapping information for county name and code (grantheader.sba1222countyname and grantheader.sba1222countycode), but we're not sure where/how those map to the DATA Act schema.
15. We need some clarification for awardee/recipient address.  
    * Will SAM be the authoritative source? The current mapping doc shows SAM for addressline1, but then maps it to a Prism table (VENADDRESS.ADDRESS1). The rest of the address fields are mapped to JAAMS (AP_SUPPLIER_SITES)  
    * The mapping doc specifies JAAMS table AP_SUPPLIER_SITES, but our data only has a table called AP_SUPPLIER_SITES_ALL. We used this--is that correct?
    * If we should, in fact, be getting address info from AP_SUPPLIER_SITES_ALL: Wwhen there are multiplier SUPPLIER_SITES records for a single vendor, which one should we use to get the address info?
16. Several DATA Act elements are mapped to JAAMS PO_LINES_ALL.QUANTITY * PO_LINES_ALL.UNIT_PRICE. Is there any additional nuance we should know about?  
    * funding action obligation
    * non-federal funding amount
    * current total funding obligation amount on award
    * current total value of award
    * potential total value of award


## JAAMS PO_HEADERS_ALL to Prism Header.docnum

This matches 18 records between JAAMS and Prism (a single award id: SBAHQ-14-S-0001)

In [565]:
header = prism['header']
#clean up header data
header = header.dropna(subset = ['header.docnum', 'header.dockey'])
header['header.dockey'] = header['header.dockey'].astype(np.int64)
header['header.verkey'] = header['header.verkey'].astype(np.int64)
header['header.docnum_stripped'] = header['header.docnum'].replace('-', '', regex = True)

po_headers_all = jaams['po_headers_all']
po_headers_all['po_headers_all.segment1_stripped'] = po_headers_all['po_headers_all.segment1'].replace('-', '', regex = True)

jp_merge = pd.merge(
    po_headers_all,
    header,
    left_on = 'po_headers_all.segment1_stripped',
    right_on = 'header.docnum_stripped'
)
print ('Number of po_headers_all records matched to prism header: {}'.format(len(jp_merge.index)))
print ('Award ids matched: {}'.format(pd.unique(jp_merge['po_headers_all.segment1'].ravel())))

Number of po_headers_all records matched to prism header: 18
Award ids matched: ['SBAHQ-14-S-0001']


## Merge in Other Required Data

### GRANTHEADER (POP congressional district)

Join columns = dockey, verkey  
1:1?  
via data/prism/joins.txt

In [566]:
#a lot of mucking around to make sure first two columns aren't interpreted as the DataFrame
#index (due to the bad file formatting...missing quote)
grantheader = pd.read_csv('data/prism/grantheader.txt', error_bad_lines = False, warn_bad_lines = False, header = None)
headers = grantheader.ix[0]
grantheader.columns = headers
grantheader = grantheader[1:]
grantheader.rename(columns = lambda x: 'grantheader.' + x.lower(), inplace = True)
grantheader['grantheader.dockey'] = grantheader['grantheader.dockey'].astype(np.int64)
grantheader['grantheader.verkey'] = grantheader['grantheader.verkey'].astype(np.int64)
jp_merge = pd.merge(
    jp_merge, grantheader,
    left_on = ['header.dockey', 'header.verkey'],
    right_on = ['grantheader.dockey', 'grantheader.verkey']
)
len(jp_merge.index)

18

### Vendor (Prism)
According to mapping doc, the first address line of the awardee/recipient comes from SAM. The remaining portions of the address are mapped to Prism table docvendor.

Join columns = dockey, verkey     
1:1?  
via data/prism/joins.txt  

In [567]:
docvendor = pd.read_csv(
    'data/prism/docVendor.txt',
    index_col = False,
    error_bad_lines = False,
    warn_bad_lines = False
)
docvendor.rename(columns = lambda x: 'docvendor.' + x.lower(), inplace = True)
jp_merge = pd.merge(
    jp_merge, docvendor,
    left_on = ['header.dockey', 'header.verkey'],
    right_on = ['docvendor.dockey', 'docvendor.verkey']
)
len(jp_merge.index)

18

### JAAMS Vendor (awardee/recipient location)

Join columns: vendor_id, vendor_site_id  
1:1?  
This join info wasn't explicitly stated in data/jaams/sql/basic table joins.sql. Just guessing here.

In [568]:
ap_supplier_sites_all = pd.read_csv('data/jaams/AP_SUPPLIER_SITES_ALL.txt')
ap_supplier_sites_all.rename(columns = lambda x: 'ap_supplier_sites_all.' + x.lower(), inplace = True)
jp_merge = pd.merge(
    jp_merge, ap_supplier_sites_all,
    left_on = ['po_headers_all.vendor_id', 'po_headers_all.vendor_site_id'],
    right_on = ['ap_supplier_sites_all.vendor_id', 'ap_supplier_sites_all.vendor_site_id'] 
)
len(jp_merge.index)

18

### PO_LINES_ALL (for award amount info)

Join columns = po_header_id  
1:n  
via data/jaams/sql/basic table joins.sql

In [569]:
po_lines_all = jaams['po_lines_all']
jp_merge = pd.merge(
    jp_merge, po_lines_all,
    left_on = ['po_headers_all.po_header_id'],
    right_on = ['po_lines_all.po_header_id']
)
len(jp_merge.index)

162

### PO_DISTRIBUTIONS_ALL (for type of transaction)

Join columns = po_header_id, po_line_id  
1:1 (i.e., in this sample data, at least, there was not more than one po distribution per po line)  
via data/jaams/sql/basic table joins.sql

In [570]:
po_distributions_all = pd.read_csv('data/jaams/PO_DISTRIBUTIONS_ALL.txt')
po_distributions_all.rename(columns = lambda x: 'po_distributions_all.' + x.lower(), inplace = True)
jp_merge = pd.merge(
    jp_merge, po_distributions_all,
    left_on = ['po_lines_all.po_header_id', 'po_lines_all.po_line_id'],
    right_on = ['po_distributions_all.po_header_id', 'po_distributions_all.po_line_id']
)
len(jp_merge.index)

162

### GL_CODE_COMBINATIONS (funding office, object class, appropriations account)

Join columns = code_combination_id  
1:1 (i.e., in this sample data, there was one gl_code_combination per po_line)  
via data/jaams/sql/basic table joins.sql

In [571]:
gl_code_combinations = pd.read_csv('data/jaams/GL_CODE_COMBINATIONS.txt')
gl_code_combinations.rename(columns = lambda x: 'gl_code_combinations.' + x.lower(), inplace = True)
jp_merge = pd.merge(
    jp_merge, gl_code_combinations,
    left_on = ['po_distributions_all.code_combination_id'],
    right_on = ['gl_code_combinations.code_combination_id']
)
len(jp_merge.index)

162

### FAADSCIV (record type, place of performance info)

faadsciv joins to the header table by using a crosswalk (called Association).  
See data/prism/joins.txt for more information  
1:1  

In [572]:
#first merge in Association so we can map faadsciv back to header
association = pd.read_csv('data/prism/Association.txt')
association.rename(columns = lambda x: 'association.' + x.lower(), inplace = True)
jp_merge = pd.merge(
    jp_merge, association,
    left_on = ['header.dockey', 'header.verkey'],
    right_on = ['association.dockey', 'association.verkey']
)
#then use Association as the crosswalk
faadsciv = pd.read_csv('data/prism/faadsciv.csv')
faadsciv.rename(columns = lambda x: 'faadsciv.' + x.lower(), inplace = True)
jp_merge = pd.merge(
    jp_merge, faadsciv,
    left_on = ['association.assocdockey', 'association.assocverkey'],
    right_on = ['faadsciv.dockey', 'faadsciv.verkey']
)
len(jp_merge.index)

162

### Prism: Docaddr (awarding office name)

Join columns: header.issuingdocaddresskey = docaddr.docaddrkey  
1:?  
via data/prism/joins.txt

#### Docaddr is missing, so hard-coding it based on the provided file: data_act_prism_grants_fy14.csv

In [573]:
jp_merge['docaddr.name'] = 'Office of Grants Management'

### Prism: Vendor2 (recipient type)

Join columns: ??  
1:??  
data/prism/joins.txt does not include Vendor2

#### Vendor2 is missing, so hard-coding it based on the provided file: data_act_prism_grants_fy14.csv

In [574]:
jp_merge['vendor2.businesstype'] = 'Other nonprofit'

### JAAMS: FIND_FLEX_VALUES_VL (funding office code)

Join columns: ??  
1:??  
data/jaams/sql/basic table joins.sql does not include find_flex_values_vl

#### FIND_FLEX_VALUES_VL is missing, so hard-coding based on provided file: data_act_prism_grants_fy14.csv

In [575]:
jp_merge['find_flex_values_vl'] = '602001'

## Add Calculated Fields and Various Hard-Coding

In [576]:
jp_merge['po_lines_all.total_amount'] = jp_merge['po_lines_all.quantity'] * jp_merge['po_lines_all.unit_price']
jp_merge['funding_agency_name'] = 'Small Business Administration'
jp_merge['funding_agency_code'] = '073'
jp_merge['funding_sub_tier_agency_name'] = 'Small Business Administration'
jp_merge['funding_sub_tier_agency_code'] = '073'
jp_merge['awarding_agency_name'] = 'Small Business Administration'
jp_merge['awarding_agency_code'] = '073'
jp_merge['awarding_sub_tier_agency_name'] = 'Small Business Administration'
jp_merge['awarding_sub_tier_agency_code'] = '073'
jp_merge['federal_agency'] = 'Small Business Administration'
jp_merge['tas'] = '730100' #this is what was listed in itemacct.tas#

## Reduce the Huge Merged File to DATA Act Elements

In [577]:
#jp_merge = jp_merge.drop_duplicates()
data_act = jp_merge[[
    'po_lines_all.item_description', #award description
    'po_headers_all.segment1', #award id
    'header.issuingdocaddresskey', #awarding office code
    'header.awarddate', #action date
    'docvendor.name', #Recipient name
    'po_distributions_all.attribute10', #period of performance start date
    'po_distributions_all.attribute11', #period of performance end date
    'docvendor.duns', #awardee/recipient legal business DUNS
    'docvendor.dunsplus4', #awardee/recipient legal business DUNS+4
    'docvendor.address1', #awardee/recipient legal business street address line 1
    'ap_supplier_sites_all.address_line1', #just including this in to see if it matches address above
    'ap_supplier_sites_all.address_line2', #awardee/recipient legal business street address line 2
    'ap_supplier_sites_all.address_line3', #awardee/recipient legal business street address line 3
    'ap_supplier_sites_all.city', #awardee/recipient legal business city
    'ap_supplier_sites_all.state', #awardee/recipient state
    'ap_supplier_sites_all.zip', #awardee/recipient us zip code + 4; awardee/recipient postal code
    'grantheader.sba1222countyname', #recipient county name
    'grantheader.sba1222countycode', #recipient county code
    'po_lines_all.quantity',
    'po_lines_all.unit_price',
    'po_lines_all.total_amount', #funding action obligation (does not account for cancellations etc.)
    'funding_agency_name', #funding agency name
    'funding_agency_code', #funding agency code
    'funding_sub_tier_agency_name', #funding sub-tier agency name
    'funding_sub_tier_agency_code', #funding sub-tier agency code
    'po_distributions_all.attribute_category', #type of award code
    'header.obligatedamt', #amount of ba appropriated; obligation
    'po_distributions_all.quantity_billed', #outlay
    'gl_code_combinations.segment3', #funding office name
    'gl_code_combinations.segment5', #object class woo!
    'gl_code_combinations.code_combination_id', #appropriations account
    'gl_code_combinations.segment4', #program activity
    'grantheader.sba1222congdistno', #place of performance congressional district
    'faadsciv.recordtype', #record type
    'faadsciv.countycityname', #primary place of performance city code (note: there was no census_code columns)
    'faadsciv.principalstatecode', #primary place of performance state code
    'faadsciv.principalstatename', #primary place of performance state name
    'faadsciv.placeofperfzip', #primary place of performance zip code + 4
    'faadsciv.placeofperfcountrycode', #primary location of performance country code
    'faadsciv.placeofperfcountryname', #primary location of performance country name
    'faadsciv.cfdaprogramnumber', #cfda program number
    'faadsciv.cfdaprogramtitle', #cfda program title
    'docaddr.name', #awarding office name
    'vendor2.businesstype', #recipient type
    'funding_agency_name',
    'funding_agency_code',
    'funding_sub_tier_agency_name',
    'funding_sub_tier_agency_code',
    'awarding_agency_name',
    'awarding_agency_code',
    'awarding_sub_tier_agency_name',
    'awarding_sub_tier_agency_code',
    'federal_agency'
    ]]

#write out the data act file
data_act.to_csv('data/data_act.csv', index = False)
#also write out the entire merged file, so we can look for interesting things
jp_merge.to_csv('data/jp_merge.csv', index = False)